# How One-Time Passwords (OTP) Work

## Cryptographic Hash Algorithm (Secure Hash Algorithm, SHA)


Hash algorithms can make fixed-length "fingerprint" out of any data. It's one-way function: the same input will always generate the same output, but it's impossible to get the original input from the hash. The hashes are not unique, there are multiple inputs that have the same hash, but in case of good algorithm we it's impossible to find the equivalent inputs without brute forcing that.

Let's have the following text:

In [1]:
var text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nunc euismod, lacus elementum "
    + "ultricies egestas, felis nibh congue mi, in porta enim ante eget diam. Nunc malesuada "
    + "tincidunt quam, at viverra risus malesuada id. Quisque a ligula laoreet, porttitor ex non, "
    + "euismod lacus. In velit odio, placerat et ultrices at, mattis sit amet tellus. Aliquam erat "
    + "volutpat. Aliquam mattis ligula urna, at ultricies purus molestie ac. Praesent ornare dignissim erat.";

Next we'll create a function that will convert the text to bytes (hash functions work with bytes, not text), compute SHA-256 hash and encode it using Base16 to text:

In [2]:
static string ComputeSha256Hash(string input) {
    // Convert input string to bytes
    var inputBytes = Encoding.UTF8.GetBytes(input);

    // Compute hash using SHA-256
    var hashBytes = System.Security.Cryptography.SHA256.HashData(inputBytes);

    // Convert hash bytes to Base16 string
    var hashString = string.Join(string.Empty,  hashBytes.Select(b => b.ToString("x2")));

    return hashString;
}

Let's see the hash:

In [3]:
ComputeSha256Hash(text)

ecc8f47d5608d89f498efefbb61d8978a8d0667733562ea91c0b43f4dfb525fb

Now we change the input text just by one bit - replace the initial `L` with `M`:

In [4]:
var text2 = "Morem ipsum dolor sit amet, consectetur adipiscing elit. Nunc euismod, lacus elementum "
    + "ultricies egestas, felis nibh congue mi, in porta enim ante eget diam. Nunc malesuada "
    + "tincidunt quam, at viverra risus malesuada id. Quisque a ligula laoreet, porttitor ex non, "
    + "euismod lacus. In velit odio, placerat et ultrices at, mattis sit amet tellus. Aliquam erat "
    + "volutpat. Aliquam mattis ligula urna, at ultricies purus molestie ac. Praesent ornare dignissim erat.";
ComputeSha256Hash(text2)

a1d54f02439d82e09f3f8fbf9949ad01eb2b06639f5304ff82b847d42fe21e45

See? Properties of the secure hash algorithms ensure that even the smallest change of input causes dramatic change of the ouput. This is exploited by the one-time password generating standards.

## Hash-based One-Time Password (HOTP)


This is defined by the [RFC 4226](https://datatracker.ietf.org/doc/html/rfc4226) specification. Let's have a secret shared between server and user, it's called a _seed_:

In [5]:
var otpSeed = "THIS_IS_MY_SECRET_SEED";

Then define a counter. Each side keeps counter how many one-time passwords was generated (client software) and how many has been already used (server):

In [6]:
var otpCounter = 1;

Every time we need a new one-time password, we simply increment the counter, add it to a seed and compute hash of it. The result is our one-time password:

In [7]:
ComputeSha256Hash(otpSeed + "|" + otpCounter)

17c44088beb41aedda1b721fd9a7842cdd4c70831fdcf3ab934e28ee17309f90

Let's generate some more passwords by incrementing the counter:

In [8]:
for(var counter = 1; counter < 10; counter++) {
    var otp = ComputeSha256Hash(otpSeed + "|" + counter);
    Console.WriteLine($"Password #{counter} is {otp}");
}

Password #1 is 17c44088beb41aedda1b721fd9a7842cdd4c70831fdcf3ab934e28ee17309f90
Password #2 is a5d7058c2e659ce983972131d13d180d223b6b668bc8086aa9dc2cfb0b79029a
Password #3 is 8e801fee682a4dc6597a68d976e886cd45580dd7d8edc82c8d261b9438f3c885
Password #4 is ffc885e4ce93a746d711abaf69559ec8cdb68fbcbc65bdf8577c3ffab1d7a6c6
Password #5 is d83acf609a7dd4656ccd796c1366ead11f81070716a8d12b50646df7f8ed999c
Password #6 is f87834a7d908f0501c205575fc1edd6949acb5b78f5ff2f4fbf19b4aa200fd69
Password #7 is 94279bb4f2316082a51435815fd7ffbc65f6da202dd226c325b83f67b415aab9
Password #8 is 7124e1ec0f634a57faf11eea4e36c8e2965f13db5478bb738b0a05f55ff51a1a
Password #9 is 44fa1bc815585841eee911f0df917189cca8f9e5e988455b824982ded7729a13


_**Note:** Real OTP generators do use bit more complicated method to join the seed and the counter together. Also, the password is (usually) just six decimal numbers, instead of full hash. There is a clever method how to extract the numbers from the raw hash data. If you're interested how it all works, look into the RFC, there's example code inside._

## Time-based One-Time Passwords (TOTP)


The above method works just fine, but it has one drawback: the generating application and server has to keep synchronized counter of generated passwords and their uses. If the counter gets mismatched, entire system would not work. There are some workarounds, like that server would try several more counter values in case some request will be lost, but it's a hassle.

To solve this problem [RFC 6238](https://datatracker.ietf.org/doc/html/rfc6238) defines time-based OTP. It's exactly the same thing as above, but instead of incrementing counter we use number of 30-second intervals elapsed since midnight of January 1, 1970 (usually called _Unix time_). Therefore, server and client need only to have reasonably accurate clocks (and again, usually few codes before and after the current one are accepted).

Let's compute number of those intervals for current time and derive the appropriate one-time password:

In [9]:
var seconds = DateTimeOffset.UtcNow.ToUnixTimeSeconds();
var intervals = seconds / 30;
var totp = ComputeSha256Hash(otpSeed + "|" + intervals);

Console.WriteLine($"Password for interval #{intervals} is {totp}");

Password for interval #56354987 is 80f8a09f62843adf642cf5591973161dccbe3ff48d29db9395ef35c0f5c84172
